In [251]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import math

In [252]:
pd.set_option("display.max_rows", 1000)

# Preprocessing

In [335]:
# raw_path = '../data/raw'
# train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
# test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [336]:
df = pd.read_csv('train.csv')
val = pd.read_csv('test.csv')

In [337]:
TARGET_COL = ['SalePrice']

ID_COL = 'Id'

REAL_COLS = ['MSSubClass', 'OverallQual', 'OverallCond', 'LotFrontage', 'LotArea',
             'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
             '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
             'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
             'TotRmsAbvGrd','Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
             'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

CAT_COLS = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
            'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
            'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
            'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
            'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
            'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
            'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
            
DATA_COLS = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']


##for function delete_nan_value
BASEMENT_COLS = ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
MASONRY_COLS = ['MasVnrArea']
GARAGE_COLS =['GarageFinish', 'GarageQual', 'GarageCond']
GARAGE_COL = ['GarageYrBlt']

MAX_NUMBER_NAN_VALUE_COLS = ['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'BsmtQual', 'GarageType' , 'FireplaceQu']
NUMBER_OF_TEST = 15



In [338]:
def preprocess_target(df: pd.DataFrame) -> pd.DataFrame:
    df[TARGET_COL] = df[TARGET_COL].astype(np.int32)
    return df

def extract_target(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df, target = df.drop(TARGET_COL, axis = 1), df[TARGET_COL]
    return df, target

def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')
    df[REAL_COLS] = df[REAL_COLS].astype(np.int32)
    df[DATA_COLS] = df[DATA_COLS].astype(np.int32)
    return df

def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df


In [339]:
def delete_nan_value(df: pd.DataFrame) ->pd.DataFrame:
    ##basement 
    for col in BASEMENT_COLS:
        temp_array = list(df[col])
        count = 0
        for index, row in df.iterrows():
            if row['BsmtQual'] == 'NA':
                if pd.isna(row[col]):
                    temp_array[count] = 'NA'
            count += 1
        df = df.drop(columns = col)
        count = 0
        for index, row in df.iterrows():
            df.loc[index,col] = temp_array[count]
            count += 1 
        df = df[(pd.isna(df[col])) == False] 
    
    ##Masonry veneer
    for col in MASONRY_COLS:
        temp_array = list(df[col])
        count = 0
        for index, row in df.iterrows():
            if row['MasVnrType'] == 'None':
                if pd.isna(row[col]):
                    temp_array[count] = 0
            count += 1
        df = df.drop(columns = col)
        count = 0
        for index, row in df.iterrows():
            df.loc[index,col] = temp_array[count]
            count += 1 
        df = df[(pd.isna(df[col])) == False] 
    
    ##Electrical 
    df = df[(pd.isna(df['Electrical'])) == False] 

    ##Garage
    for col in GARAGE_COLS:
        temp_array = list(df[col])
        count = 0
        for index, row in df.iterrows():
            if row['GarageType'] == 'NA':
                if pd.isna(row[col]):
                    temp_array[count] = 'NA'
            count += 1
        df = df.drop(columns = col)
        count = 0
        for index, row in df.iterrows():
            df.loc[index,col] = temp_array[count]
            count += 1 
        df = df[(pd.isna(df[col])) == False] 
    for col in GARAGE_COL:
        temp_array = list(df[col])
        count = 0
        for index, row in df.iterrows():
            if row['GarageType'] == 'NA':
                if pd.isna(row[col]):
                    temp_array[count] = 0
            count += 1
        df = df.drop(columns = col)
        count = 0
        for index, row in df.iterrows():
            df.loc[index,col] = temp_array[count]
            count += 1 
        df = df[(pd.isna(df[col])) == False]            
    return df

In [340]:
def full_columns_with_max_number_of_nan(df: pd.DataFrame) -> pd.DataFrame:
    df[MAX_NUMBER_NAN_VALUE_COLS] = df[MAX_NUMBER_NAN_VALUE_COLS].replace(np.nan, 'NA')
    df['LotFrontage'] = df['LotFrontage'].replace(np.nan, 0) 
    df['MasVnrType'] = df['MasVnrType'].replace(np.nan, 'None')

    return df


In [341]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = full_columns_with_max_number_of_nan(df)
    df = delete_nan_value(df)
    df = cast_types(df)
    return df

In [342]:
def preprocess_val(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = full_columns_with_max_number_of_nan(df)
    df = delete_nan_value(df)
    df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
    df = cast_types(df)
    return df

In [343]:
import pickle
from typing import Union
from pandas import DataFrame
from pandas.core.indexes.base import Index as PandasIndex


def save_as_pickle(obj: Union[DataFrame, PandasIndex], path: str) -> None:
    if isinstance(obj, DataFrame):
        obj.to_pickle(path)
    elif isinstance(obj, PandasIndex):
        with open('path', 'wb') as f:
            pickle.dump(obj, f)

In [344]:
df = preprocess_df(df)
val = preprocess_val(val)

##только для train()
df, target = extract_target(df)
target = preprocess_target(target)
# save_as_pickle(target, '../data/processed/target_for_data_for_train.pkl')
# save_as_pickle(df, '../data/interim/data_for_train.pkl')

# ##только для val
# save_as_pickle(val, '../data/interim/val.pkl')

# Feature generation

In [345]:
# interim_data_path = '../data/interim/'
# train = pd.read_pickle(os.path.join(interim_data_path, 'data_for_train.pkl'))
# test = pd.read_pickle(os.path.join(interim_data_path, 'val.pkl'))

In [346]:
train = df
test = val

In [347]:
##for feature generation  class of building

def LotFrontage_points(lotfrontage_) -> int:
    if lotfrontage_ >= 100:
        return 5
    elif 40 <= lotfrontage_ < 100:
        return 4
    elif 30 <= lotfrontage_ < 40:
        return 3
    elif 0 < lotfrontage_ < 30:
        return 2 
    else:
        return 0

def LotArea_points(lotarea_) -> int:
    if lotarea_ >= 16000:
        return 5
    elif 9000 <= lotarea_ < 16000:
        return 4
    elif 5000 <= lotarea_ < 9000:
        return 3
    elif 0 < lotarea_ < 5000:
        return 2
    else:
        return 0

def Utilities_points(utilities_) -> int:
    if utilities_ == 'AllPub':
        return 5
    elif utilities_ == 'NoSewr':
        return 4
    elif utilities_ == 'NoSeWa':
        return 3
    elif utilities_ == 'ELO':
        return 2 
    else:
        return 0

def OverallQual_points(overallqual_) -> int:
    excellent = [10, 9, 8]
    good = [7, 6]
    average = [5, 4]
    poor = [3, 2, 1]
    if overallqual_ in excellent:
        return 5
    elif overallqual_ in good:
        return 4
    elif overallqual_ in average:
        return 3
    elif overallqual_ in poor:
        return 2
    else:
        return 0

def OverallCond_points(overallcond_) -> int:
    excellent = [10, 9, 8]
    good = [7, 6]
    average = [5, 4]
    poor = [3, 2, 1]
    if overallcond_ in excellent:
        return 5
    elif overallcond_ in good:
        return 4
    elif overallcond_ in average:
        return 3
    elif overallcond_ in poor:
        return 2
    else:
        return 0

def YearBuilt_points(yearbuilt_) -> int:
    if yearbuilt_ >= 2000:
        return 5
    elif 1971 <= yearbuilt_ < 2000:
        return 4
    elif 1920 <= yearbuilt_ < 1971:
        return 3
    elif yearbuilt_ < 1920:
        return 2
    else:
        return 0

def BsmtFinType1_points(bsmtfintype1_) -> int:
    if bsmtfintype1_ == 'GLQ' or bsmtfintype1_ == 'ALQ':
        return 5
    elif bsmtfintype1_ == 'BLQ' or bsmtfintype1_ == 'Rec':
        return 4
    elif bsmtfintype1_ == 'LwQ':
        return 3
    elif bsmtfintype1_ == 'Unf':
        return 2 
    else:
        return 0

def BsmtFinType2_points(bsmtfintype2_) -> int:
    if bsmtfintype2_ == 'GLQ' or bsmtfintype2_ == 'ALQ':
        return 5
    elif bsmtfintype2_ == 'BLQ' or bsmtfintype2_ == 'Rec':
        return 4
    elif bsmtfintype2_ == 'LwQ':
        return 3
    elif bsmtfintype2_ == 'Unf':
        return 2 
    else:
        return 0

def TotalBsmtSF_points(totalbsmtsf_) -> int:
    if totalbsmtsf_ >= 3000:
        return 5
    elif 1000 <= totalbsmtsf_ < 3000:
        return 4
    elif 500 <= totalbsmtsf_ < 1000:
        return 3
    elif 0 < totalbsmtsf_ < 500:
        return 2
    else:
        return 0

def TotRmsAbvGrd_points(totrmsabvgrd_) -> int:
    if totrmsabvgrd_ >= 10:
        return 5
    elif 6 <= totrmsabvgrd_ < 10:
        return 4
    elif 3 <= totrmsabvgrd_ < 6:
        return 3
    elif totrmsabvgrd_ < 3:
        return 2
    else:
        return 0

def GarageType_points(garagetype_) -> int:
    if garagetype_ == '2Types' or garagetype_ == 'Attchd':
        return 5
    elif garagetype_ == 'Basment' or garagetype_ == 'BuiltIn':
        return 4
    elif garagetype_ == 'CarPort':
        return 3
    elif garagetype_ == 'Detchd':
        return 2 
    else:
        return 0

def GarageYrBlt_points(garageyrblt_) -> int:
    if garageyrblt_ >= 2000:
        return 5
    elif 1971 <= garageyrblt_ < 2000:
        return 4
    elif 1920 <= garageyrblt_ < 1971:
        return 3
    elif garageyrblt_ < 1920:
        return 2
    else:
        return 0

def GarageArea_points(garagearea_) -> int:
    if garagearea_ >= 900:
        return 5
    elif 400 <= garagearea_ < 900:
        return 4
    elif 200 <= garagearea_ < 400:
        return 3
    elif 0 < garagearea_ < 200:
        return 2
    else:
        return 0

def PoolArea_points(poolarea_) -> int:
    if poolarea_ >= 100:
        return 5
    elif 50 <= poolarea_ < 100:
        return 4
    elif 2 <= poolarea_ < 50:
        return 3
    elif 0 < poolarea_ < 2:
        return 2
    else:
        return 0

def MiscFeature_points(miscfeature_) -> int:
    if miscfeature_ != 'NA':
        return 5
    else:
        return 0


In [348]:
def class_of_building(df: pd.DataFrame) -> pd.DataFrame:
    building_class = []
    for index, row in df.iterrows():
        count = 0
        count += LotFrontage_points(row['LotFrontage'])
        count += LotArea_points(row['LotArea'])
        count += Utilities_points(row['Utilities'])
        count += OverallQual_points(row['OverallQual'])
        count += OverallCond_points(row['OverallCond'])
        count += YearBuilt_points(row['YearBuilt'])
        count += BsmtFinType1_points(row['BsmtFinType1'])
        count += BsmtFinType2_points(row['BsmtFinType2'])
        count += TotalBsmtSF_points(row['TotalBsmtSF'])
        count += TotRmsAbvGrd_points(row['TotRmsAbvGrd'])
        count += GarageType_points(row['GarageType'])
        count += GarageYrBlt_points(row['GarageYrBlt'])
        count += GarageArea_points(row['GarageArea'])
        count += PoolArea_points(row['PoolArea'])
        count += MiscFeature_points(row['MiscFeature']) 
        building_class.append(round( count/NUMBER_OF_TEST))
    i = 0 
    for index, row in df.iterrows():
            df.loc[index,'ClassBuilt'] = building_class[i]
            i += 1 
    df['ClassBuilt'] = df['ClassBuilt'].astype(np.int32)
    return df
    

In [349]:
## age at the sold moment 
def age_of_building_at_the_time_of_sale(df: pd.DataFrame) -> pd.DataFrame:
    building_age = []
    for index, row in df.iterrows():
        building_age.append(int(row['YrSold'] - row['YearBuilt']))
    i = 0 
    for index, row in df.iterrows():
            df.loc[index,'BuiltAge'] = building_age[i]
            i += 1 
    df['BuiltAge'] = df['BuiltAge'].astype(np.int32)
    return df

In [350]:
def feature_generation(df: pd.DataFrame) -> pd.DataFrame:
    df = class_of_building(df)
    df = age_of_building_at_the_time_of_sale(df)
    return df

In [351]:
train = feature_generation(train)
test = feature_generation(test)

# save_as_pickle(test, '../data/processed/test.pkl')
# save_as_pickle(train, '../data/processed/data_for_train.pkl')


# Modeling

In [352]:
import os
import pandas as pd
import numpy as np

from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, explained_variance_score

#%pip install catboost
import catboost
from catboost import CatBoostRegressor

#%pip install category_encoders
import category_encoders as ce

#%pip install xgboost
import xgboost 

## Training

In [367]:
##param 
#catboost
iterations_ = 1000
max_depth_ = 4
ctr_leaf_count_limit_ = 20  ##param to control cat features 
loss_function_ = 'RMSE'
learning_rate_ = 0.2

#xgboost
booster_ = 'dart'
eta_ = 0.2
tree_method_ =  'exact'



In [270]:
# processed_data_path = '../data/processed/'
# train = pd.read_pickle(os.path.join(processed_data_path, 'data_for_train.pkl'))
# target = pd.read_pickle(os.path.join(processed_data_path, 'target_for_data_for_train.pkl'))

In [357]:
##удалить и использовать блок выше, так как там через папки
train_ = train
target = target
test = test

In [358]:

def print_metric(y_true, y_predict):
    print('R2: ', r2_score(y_true, y_predict))
    print('MSE: ', mean_squared_error(y_true, y_predict))
    print('MAPE: ', mean_absolute_percentage_error(y_true, y_predict))
    print('Explained variance', explained_variance_score(y_true, y_predict))

In [359]:
X_train, X_test, Y_train, Y_test = train_test_split(train, target, test_size=0.2, random_state=77)

In [360]:
##CATBOOST
model = CatBoostRegressor(
    iterations = iterations_,
    max_depth = max_depth_,
    ctr_leaf_count_limit = ctr_leaf_count_limit_, ##param to control cat features 
    loss_function = loss_function_,
    learning_rate = learning_rate_, 
    cat_features = CAT_COLS)

pipeline_castboost = Pipeline ([
    ('model_cast', model)])

pipeline_castboost.fit(X_train,Y_train)

c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 67483.2708574	total: 36.9ms	remaining: 36.8s
1:	learn: 60492.1829426	total: 65.6ms	remaining: 32.7s
2:	learn: 54777.7669820	total: 86.3ms	remaining: 28.7s
3:	learn: 50328.2656598	total: 108ms	remaining: 26.9s
4:	learn: 46340.4430431	total: 132ms	remaining: 26.3s
5:	learn: 42869.3612441	total: 154ms	remaining: 25.5s
6:	learn: 40375.9464095	total: 176ms	remaining: 25s
7:	learn: 38021.9164672	total: 200ms	remaining: 24.8s
8:	learn: 35979.9499825	total: 230ms	remaining: 25.3s
9:	learn: 34475.1499573	total: 267ms	remaining: 26.4s
10:	learn: 33116.7234469	total: 296ms	remaining: 26.6s
11:	learn: 31976.3281376	total: 332ms	remaining: 27.4s
12:	learn: 30804.3884680	total: 360ms	remaining: 27.3s
13:	learn: 29920.5352128	total: 383ms	remaining: 27s
14:	learn: 28971.9031606	total: 419ms	remaining: 27.5s
15:	learn: 28215.8750198	total: 453ms	remaining: 27.9s
16:	learn: 27461.7583703	total: 536ms	remaining: 31s
17:	learn: 26938.7826241	total: 579ms	remaining: 31.6s
18:	learn: 26355.347796

Pipeline(steps=[('model_cast',
                 <catboost.core.CatBoostRegressor object at 0x0000028C49B0F4C0>)])

In [361]:
y_predict = pipeline_castboost.predict(X_test)
print_metric(Y_test, y_predict)

R2:  0.9128876090777809
MSE:  700443766.1059676
MAPE:  0.08854580094386787
Explained variance 0.9139384129219799


In [368]:
real_pipe = Pipeline([
    ('scaler', StandardScaler())])

cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore',sparse=False))
])

preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, REAL_COLS),
    ('cat_cols', cat_pipe, CAT_COLS),
    ('cat_bost_cols', ce.CountEncoder(), CAT_COLS), ##for work with cat_features
]
)

model = xgboost.XGBRegressor(
    booster = booster_,
    eta = eta_,
    tree_method = tree_method_
)

model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model)
]
)

pipline_xg = model_pipe
pipline_xg.fit(X_train, Y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('real_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'OverallQual',
                                                   'OverallCond', 'LotFrontage',
                                                   'LotArea', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBat...
                              eval_metric=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.200000003, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0, ...))])

In [369]:
y_predict = pipline_xg.predict(X_test)
print_metric(Y_test, y_predict)

R2:  0.8932990166778475
MSE:  857949573.1222739
MAPE:  0.09536305850928414
Explained variance 0.8937712746708963


In [278]:
#Раскоментировать для работы с папками!
# from joblib import dump, load
# path_to_model = '../models/'
# dump(pipeline_castboost, os.path.join(path_to_model, 'pipeline_castboost.joblib')) 
# dump(pipline_xg, os.path.join(path_to_model, 'pipline_xg.joblib')) 

## Inference

In [279]:
# # #processed_data_path = '../data/processed/'
# # #path_to_model = '../models/'
# test = pd.read_pickle(os.path.join(raw_path, 'test.csv'))
# model_1  = load(os.path.join(path_to_model, 'pipeline_castboost.joblib')) 
# model_2 = load(os.path.join(path_to_model, 'pipline_xg.joblib')) 

In [370]:
##удалить и использовать блок выше, так как там через папки
model_1 = pipeline_castboost
model_2 = pipline_xg

In [372]:
y_pred_1  = model_1.predict(test)
print(y_pred_1)
y_pred_2  = model_2.predict(test)
print(y_pred_2)

c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[127299.71562268 161724.71411287 192422.76566673 ... 172961.63834093
 111179.68556311 222365.1231344 ]
[125808.44 160586.78 188681.94 ... 158962.94 111972.99 240630.61]
